In [ ]:
# RUN THIS FIXED VERSION WITH RELAXED FILTERING AND BETTER DEBUGGING

# =============================================================================
# ENHANCED DATA PREPARATION WITH RELAXED FILTERING
# =============================================================================

# Install required packages
!pip install -q datasets transformers torch accelerate bitsandbytes
!pip install -q sentencepiece protobuf
!pip install -q requests beautifulsoup4

import torch
from datasets import load_dataset, DatasetDict
import pandas as pd
import json
from tqdm import tqdm
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import os
import random
from google.colab import drive
import shutil
import time

# =============================================================================
# 0. GOOGLE DRIVE SETUP & RESUME MANAGER (WITH EXTRA DEBUGGING)
# =============================================================================

class ResumeManager:
    def __init__(self, base_path="/content/drive/MyDrive/financial_llm"):
        self.base_path = base_path
        self.checkpoint_file = f"{base_path}/data/checkpoint.json"
        print(f"🔧 ResumeManager initialized with base_path: {base_path}")
        self.setup_google_drive()

    def setup_google_drive(self):
        """Mount Google Drive and create necessary directories"""
        print("🔧 Setting up Google Drive...")
        try:
            drive.mount('/content/drive', force_remount=True)  # Force remount to ensure visibility
            print("✅ Google Drive mounted successfully")
        except Exception as e:
            print(f"⚠️ Google Drive mounting: {e}")

        # Create directory structure with verification
        directories = [
            f"{self.base_path}/data",
            f"{self.base_path}/models",
            f"{self.base_path}/results"
        ]

        for directory in directories:
            os.makedirs(directory, exist_ok=True)
            # Verify creation
            if os.path.exists(directory):
                print(f"✅ Created/verified: {directory}")
            else:
                print(f"❌ Failed to create: {directory}")

    def save_checkpoint(self, stage, data=None, metadata=None):
        """Save progress checkpoint"""
        print(f"💾 Saving checkpoint for stage: {stage}")
        checkpoint = {
            'stage': stage,
            'data': data,
            'metadata': metadata,
            'timestamp': pd.Timestamp.now().isoformat()
        }

        try:
            with open(self.checkpoint_file, 'w') as f:
                json.dump(checkpoint, f, indent=2)
            print(f"✅ Checkpoint saved: {self.checkpoint_file}")
        except Exception as e:
            print(f"❌ Error saving checkpoint: {e}")

    def load_checkpoint(self):
        """Load progress checkpoint"""
        print(f"🔍 Looking for checkpoint: {self.checkpoint_file}")
        try:
            with open(self.checkpoint_file, 'r') as f:
                checkpoint = json.load(f)
            print(f"✅ Checkpoint loaded from stage: {checkpoint['stage']}")
            return checkpoint
        except FileNotFoundError:
            print("ℹ️ No checkpoint found, starting from beginning")
            return None
        except Exception as e:
            print(f"❌ Error loading checkpoint: {e}")
            return None

    def clear_checkpoint(self):
        """Clear checkpoint after successful completion"""
        try:
            if os.path.exists(self.checkpoint_file):
                os.remove(self.checkpoint_file)
                print("✅ Checkpoint cleared")
            else:
                print("ℹ️ No checkpoint to clear")
        except Exception as e:
            print(f"❌ Error clearing checkpoint: {e}")

# =============================================================================
# ENHANCED DEBUGGING FOR DATA COLLECTION
# =============================================================================

class EnhancedFinancialDataCollector:
    def __init__(self, resume_manager):
        self.resume_manager = resume_manager
        self.safety_filters = self._initialize_safety_filters()
        print("🔧 EnhancedFinancialDataCollector initialized")

    def _initialize_safety_filters(self):
        """Initialize comprehensive safety filters"""
        return {
            'strict_blockers': [
                'buy now', 'sell immediately', 'guaranteed returns',
                'insider information', 'you should invest', 'price target',
                'investment recommendation', 'hot tip', 'secret strategy',
                'unauthorized advice', 'unregistered', 'bypass regulations',
                'pyramid scheme', 'ponzi scheme', 'get rich quick',
                'double your money', 'sure profit', 'can\'t lose investment'
            ]
        }

    def load_datasets_with_resume(self):
        """Load datasets with resume capability"""
        # Check if we have checkpoint data
        checkpoint = self.resume_manager.load_checkpoint()
        if checkpoint and checkpoint['stage'] == 'datasets_loaded':
            print("🔄 Resuming from checkpoint: datasets already loaded")
            return checkpoint['data']

        print("📥 Loading multiple finance datasets...")

        datasets_to_load = [
            {
                'name': 'finance-exam-data',
                'source': "1rsh/finance-exam-data-generated",
                'split': 'train',
                'priority': 'high'
            },
            {
                'name': 'finance-alpaca',
                'source': "gbharti/finance-alpaca",
                'split': 'train',
                'priority': 'high'
            }
        ]

        all_data = []
        successful_loads = 0

        for dataset_info in datasets_to_load:
            try:
                print(f"🔄 Loading {dataset_info['name']} from {dataset_info['source']}...")
                dataset = load_dataset(dataset_info['source'], split=dataset_info['split'])
                print(f"   ✅ Loaded {len(dataset)} examples")

                converted_data = self._convert_to_common_format(dataset, dataset_info['name'])
                all_data.extend(converted_data)

                successful_loads += 1
                print(f"✅ {dataset_info['name']}: {len(converted_data)} examples after conversion")

            except Exception as e:
                print(f"❌ Failed to load {dataset_info['name']}: {e}")
                continue

        print(f"📊 Total data collected: {len(all_data)} examples from {successful_loads} datasets")

        # Save checkpoint
        self.resume_manager.save_checkpoint('datasets_loaded', all_data, {
            'successful_loads': successful_loads,
            'total_examples': len(all_data)
        })

        return all_data

    def _convert_to_common_format(self, dataset, dataset_name):
        """Convert different dataset formats to common structure"""
        converted = []
        print(f"   Converting {dataset_name} to common format...")

        for example in dataset:
            try:
                if dataset_name == 'finance-exam-data':
                    converted.append({
                        'prompt': example.get('prompt', ''),
                        'completion': example.get('completion', ''),
                        'source': dataset_name
                    })
                elif dataset_name == 'finance-alpaca':
                    converted.append({
                        'prompt': example.get('instruction', ''),
                        'completion': example.get('output', ''),
                        'source': dataset_name
                    })
            except Exception as e:
                continue

        print(f"   ✅ Converted {len(converted)} examples from {dataset_name}")
        return converted

# =============================================================================
# RELAXED QUALITY ENHANCER - MUCH MORE PERMISSIVE
# =============================================================================

class RelaxedQualityEnhancer:
    def __init__(self, resume_manager):
        self.resume_manager = resume_manager

    def enhance_quality_with_resume(self, raw_data, target_avg_score=4.0):  # Lowered target score
        """Filter and enhance data quality with RELAXED criteria"""
        # Check if we have checkpoint data
        checkpoint = self.resume_manager.load_checkpoint()
        if checkpoint and checkpoint['stage'] == 'quality_enhanced':
            print("🔄 Resuming from checkpoint: quality already enhanced")
            return checkpoint['data']

        print("🎯 Enhancing dataset quality with RELAXED criteria...")
        print(f"📊 Starting with {len(raw_data)} raw examples")

        # First pass: VERY RELAXED basic filtering
        filtered_data = self._relaxed_basic_filtering(raw_data)
        print(f"📊 After relaxed filtering: {len(filtered_data)} examples")

        if len(filtered_data) == 0:
            print("🚨 CRITICAL: No data passed filtering! Using emergency fallback...")
            filtered_data = self._emergency_keep_all(raw_data)
            print(f"📊 After emergency fallback: {len(filtered_data)} examples")

        # Second pass: Quality scoring
        scored_data = self._score_data_quality(filtered_data)

        # Third pass: Quality enhancement with VERY RELAXED thresholds
        enhanced_data = self._relaxed_enhance_quality(scored_data, target_avg_score)

        print(f"📊 Final enhanced data: {len(enhanced_data)} examples")

        # Save checkpoint
        self.resume_manager.save_checkpoint('quality_enhanced', enhanced_data, {
            'original_size': len(raw_data),
            'enhanced_size': len(enhanced_data),
            'avg_score': np.mean([item['conceptual_score'] for item in enhanced_data]) if enhanced_data else 0
        })

        return enhanced_data

    def _relaxed_basic_filtering(self, data):
        """VERY RELAXED basic filtering"""
        filtered = []

        for example in tqdm(data, desc="Relaxed Filtering"):
            prompt = str(example['prompt']).strip()
            completion = str(example['completion']).strip()

            # VERY RELAXED quality checks - only remove completely empty/broken data
            if (len(prompt) >= 3 and  # Reduced from 10 to 3
                len(completion) >= 10 and  # Reduced from 80 to 10
                len(completion.split()) >= 3):  # Reduced from 40 to 3
                filtered.append(example)

        return filtered

    def _emergency_keep_all(self, data):
        """Emergency fallback - keep almost everything"""
        filtered = []
        for example in data:
            prompt = str(example['prompt']).strip()
            completion = str(example['completion']).strip()

            # Minimal criteria - only filter completely broken data
            if len(prompt) > 0 and len(completion) > 0:
                filtered.append(example)
        return filtered

    def _score_data_quality(self, data):
        """Quality scoring with RELAXED criteria"""
        scored_data = []

        for example in tqdm(data, desc="Quality Scoring"):
            prompt = example['prompt']
            completion = example['completion']

            score = self._calculate_relaxed_score(prompt, completion)

            scored_example = example.copy()
            scored_example['conceptual_score'] = score
            scored_data.append(scored_example)

        return scored_data

    def _calculate_relaxed_score(self, prompt, completion):
        """Calculate quality score with VERY RELAXED criteria"""
        score = 0
        prompt_lower = prompt.lower()
        completion_lower = completion.lower()

        # Prompt quality (0-2 points) - RELAXED
        if any(q_type in prompt_lower for q_type in ['what is', 'explain', 'describe', 'define']):
            score += 1
        if any(q_type in prompt_lower for q_type in ['compare', 'contrast', 'difference between']):
            score += 1
        if any(q_type in prompt_lower for q_type in ['advantages', 'disadvantages', 'benefits', 'risks']):
            score += 1

        # Completion quality (0-3 points) - VERY RELAXED
        word_count = len(completion.split())
        if word_count >= 50:  # Reduced from 150
            score += 2
        elif word_count >= 25:  # Reduced from 100
            score += 1
        elif word_count >= 10:  # Added very low tier
            score += 0.5

        # Structure and clarity (0-2 points) - RELAXED
        if any(marker in completion for marker in [':', '- ', '•', '1.', 'a)']):
            score += 1
        if any(term in completion_lower for term in ['for example', 'for instance', 'specifically']):
            score += 1

        # Financial depth (0-1 point) - RELAXED
        basic_terms = ['investment', 'stock', 'bond', 'market', 'risk', 'return', 'portfolio']
        if any(term in completion_lower for term in basic_terms):
            score += 1

        return min(10, score)

    def _relaxed_enhance_quality(self, data, target_avg_score):
        """Enhance dataset quality with VERY RELAXED thresholds"""
        if not data:
            print("⚠️ WARNING: No data to enhance!")
            return []

        current_avg = np.mean([item['conceptual_score'] for item in data])
        print(f"📊 Current average score: {current_avg:.2f}")
        print(f"🎯 Target average score: {target_avg_score}")

        # Start with VERY LOW thresholds
        sorted_data = sorted(data, key=lambda x: x['conceptual_score'], reverse=True)

        # Try progressively lower thresholds - starting from 2.0
        for threshold in np.arange(2.0, 5.0, 0.5):
            filtered = [item for item in sorted_data if item['conceptual_score'] >= threshold]
            if filtered:
                filtered_avg = np.mean([item['conceptual_score'] for item in filtered])
                if filtered_avg >= target_avg_score:
                    print(f"✅ Quality threshold {threshold:.1f} gives avg {filtered_avg:.2f} with {len(filtered)} examples")
                    return filtered

        # If still too restrictive, use score >= 1.0
        best_data = [item for item in sorted_data if item['conceptual_score'] >= 1.0]
        if best_data:
            best_avg = np.mean([item['conceptual_score'] for item in best_data])
            print(f"⚠️ Using very relaxed threshold: score >= 1.0 gives avg {best_avg:.2f} with {len(best_data)} examples")
        else:
            # Last resort: use all data
            best_data = data
            best_avg = current_avg
            print(f"🚨 CRITICAL: Using all available data: avg {best_avg:.2f} with {len(best_data)} examples")

        return best_data

# =============================================================================
# CONTEST SPLITTER (SAME AS BEFORE)
# =============================================================================

class ContestSplitter:
    def __init__(self, resume_manager):
        self.resume_manager = resume_manager

    def apply_contest_split(self, data):
        """Apply contest-optimized 85-10-5 split"""
        # Check if we have checkpoint data
        checkpoint = self.resume_manager.load_checkpoint()
        if checkpoint and checkpoint['stage'] == 'split_completed':
            print("🔄 Resuming from checkpoint: split already completed")
            return checkpoint['data']['train'], checkpoint['data']['val'], checkpoint['data']['test']

        print("🎯 Applying contest-optimized split (85-10-5)...")

        # Shuffle data for better distribution
        random.shuffle(data)

        total = len(data)
        train_size = int(0.85 * total)    # 85% for training
        val_size = int(0.10 * total)      # 10% for validation
        test_size = total - train_size - val_size  # 5% for test

        train_data = data[:train_size]
        val_data = data[train_size:train_size + val_size]
        test_data = data[train_size + val_size:]

        print(f"📊 Contest-optimized split:")
        print(f"   Training: {len(train_data)} examples ({len(train_data)/total*100:.1f}%)")
        print(f"   Validation: {len(val_data)} examples ({len(val_data)/total*100:.1f}%)")
        print(f"   Test: {len(test_data)} examples ({len(test_data)/total*100:.1f}%)")

        split_data = {
            'train': train_data,
            'val': val_data,
            'test': test_data
        }

        # Save checkpoint
        self.resume_manager.save_checkpoint('split_completed', split_data, {
            'total_examples': total,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'test_size': len(test_data)
        })

        return train_data, val_data, test_data

# =============================================================================
# GOOGLE DRIVE MANAGER WITH BETTER FILE HANDLING
# =============================================================================

class GoogleDriveManager:
    def __init__(self, resume_manager):
        self.resume_manager = resume_manager
        self.base_path = resume_manager.base_path
        print(f"🔧 GoogleDriveManager initialized with base_path: {self.base_path}")

    def save_final_datasets(self, train_data, val_data, test_data, metadata):
        """Save final datasets to Google Drive with metadata"""
        print("💾 Saving datasets to Google Drive...")
        print(f"📁 Target directory: {self.base_path}/data")

        # Save datasets
        datasets_path = f"{self.base_path}/data"

        # Define file paths
        train_file = f"{datasets_path}/contest_optimized_train.json"
        val_file = f"{datasets_path}/contest_optimized_validation.json"
        test_file = f"{datasets_path}/contest_optimized_test.json"

        print(f"📄 Saving files:")
        print(f"   Train: {train_file}")
        print(f"   Validation: {val_file}")
        print(f"   Test: {test_file}")

        try:
            # Save with ensure_ascii=False to handle special characters
            with open(train_file, 'w', encoding='utf-8') as f:
                json.dump(train_data, f, indent=2, ensure_ascii=False)
            print(f"✅ Saved train data: {len(train_data)} examples")

            with open(val_file, 'w', encoding='utf-8') as f:
                json.dump(val_data, f, indent=2, ensure_ascii=False)
            print(f"✅ Saved validation data: {len(val_data)} examples")

            with open(test_file, 'w', encoding='utf-8') as f:
                json.dump(test_data, f, indent=2, ensure_ascii=False)
            print(f"✅ Saved test data: {len(test_data)} examples")

        except Exception as e:
            print(f"❌ Error saving files: {e}")
            return

        # Save metadata
        metadata['saved_timestamp'] = pd.Timestamp.now().isoformat()
        metadata_file = f"{datasets_path}/dataset_metadata.json"
        try:
            with open(metadata_file, 'w') as f:
                json.dump(metadata, f, indent=2)
            print(f"✅ Saved metadata: {metadata_file}")
        except Exception as e:
            print(f"❌ Error saving metadata: {e}")

        # Save quality report
        self._save_quality_report(train_data + val_data + test_data, datasets_path)

        # Force sync and verify files
        self._force_sync_and_verify(datasets_path)

        print("✅ All datasets saved to Google Drive")
        print(f"📁 Path: {datasets_path}")

    def _save_quality_report(self, data, save_path):
        """Generate and save quality report"""
        scores = [item['conceptual_score'] for item in data]
        sources = [item.get('source', 'unknown') for item in data]

        report = {
            'total_examples': len(data),
            'average_quality_score': np.mean(scores),
            'quality_std_dev': np.std(scores),
            'score_distribution': {
                'excellent_9_10': len([s for s in scores if s >= 9]),
                'good_7_8': len([s for s in scores if 7 <= s < 9]),
                'fair_5_6': len([s for s in scores if 5 <= s < 7]),
                'basic_3_4': len([s for s in scores if 3 <= s < 5]),
                'low_1_2': len([s for s in scores if 1 <= s < 3]),
                'minimal_0': len([s for s in scores if s < 1])
            },
            'source_distribution': {},
            'generation_timestamp': pd.Timestamp.now().isoformat()
        }

        # Source distribution
        for source in set(sources):
            report['source_distribution'][source] = sources.count(source)

        report_file = f"{save_path}/quality_report.json"
        try:
            with open(report_file, 'w') as f:
                json.dump(report, f, indent=2)
            print(f"✅ Saved quality report: {report_file}")
        except Exception as e:
            print(f"❌ Error saving quality report: {e}")

    def _force_sync_and_verify(self, datasets_path):
        """Force file sync and verify files exist"""
        print("\n🔍 Force syncing and verifying files...")

        # Force sync by listing directory
        try:
            # This forces Google Drive to sync
            time.sleep(2)  # Wait a bit for sync
            files = os.listdir(datasets_path)
            print(f"📁 Files in directory: {files}")

            # Check each expected file
            expected_files = [
                'contest_optimized_train.json',
                'contest_optimized_validation.json',
                'contest_optimized_test.json',
                'dataset_metadata.json',
                'quality_report.json'
            ]

            for file in expected_files:
                file_path = os.path.join(datasets_path, file)
                if os.path.exists(file_path):
                    file_size = os.path.getsize(file_path)
                    print(f"   ✅ {file}: {file_size} bytes")
                else:
                    print(f"   ❌ {file}: NOT FOUND")

        except Exception as e:
            print(f"❌ Error during file verification: {e}")

# =============================================================================
# MAIN EXECUTION PIPELINE WITH RELAXED FILTERING
# =============================================================================

def main_pipeline_with_relaxed_filtering():
    """Main pipeline with RELAXED filtering criteria"""
    print("🚀 ENHANCED DATA PREPARATION PIPELINE WITH RELAXED FILTERING")
    print("=" * 60)

    # Initialize managers
    resume_manager = ResumeManager()
    data_collector = EnhancedFinancialDataCollector(resume_manager)
    quality_enhancer = RelaxedQualityEnhancer(resume_manager)  # Use RELAXED enhancer
    splitter = ContestSplitter(resume_manager)
    drive_manager = GoogleDriveManager(resume_manager)

    try:
        # Step 1: Load datasets (with resume)
        print("\n📥 STEP 1: Loading datasets...")
        raw_data = data_collector.load_datasets_with_resume()

        # Step 2: Enhance quality with RELAXED criteria
        print("\n🎯 STEP 2: Enhancing dataset quality with RELAXED criteria...")
        enhanced_data = quality_enhancer.enhance_quality_with_resume(raw_data, target_avg_score=3.0)  # Lower target

        # Step 3: Apply contest-optimized split
        print("\n📊 STEP 3: Applying contest-optimized split...")
        train_data, val_data, test_data = splitter.apply_contest_split(enhanced_data)

        # Step 4: Save to Google Drive
        print("\n💾 STEP 4: Saving to Google Drive...")
        metadata = {
            'total_examples': len(enhanced_data),
            'train_examples': len(train_data),
            'val_examples': len(val_data),
            'test_examples': len(test_data),
            'avg_quality_score': np.mean([item['conceptual_score'] for item in enhanced_data]) if enhanced_data else 0,
            'split_strategy': 'contest_optimized_85_10_5',
            'filtering_strategy': 'RELAXED'
        }
        drive_manager.save_final_datasets(train_data, val_data, test_data, metadata)

        # Clear checkpoint on successful completion
        resume_manager.clear_checkpoint()

        # Final report
        print("\n✅ PIPELINE COMPLETED SUCCESSFULLY!")
        print(f"🎯 Final Dataset Stats:")
        print(f"   Total high-quality examples: {len(enhanced_data)}")
        print(f"   Average quality score: {metadata['avg_quality_score']:.2f}/10")
        print(f"   Training examples: {len(train_data)} ({len(train_data)/len(enhanced_data)*100:.1f}%)")
        print(f"   Validation examples: {len(val_data)} ({len(val_data)/len(enhanced_data)*100:.1f}%)")
        print(f"   Test examples: {len(test_data)} ({len(test_data)/len(enhanced_data)*100:.1f}%)")
        print(f"   📁 Saved to: {resume_manager.base_path}/data/")

        return train_data, val_data, test_data

    except Exception as e:
        print(f"❌ Pipeline failed: {e}")
        print("💡 Progress saved in checkpoint. Re-run to resume.")
        raise

# =============================================================================
# EXECUTION WITH BETTER VERIFICATION
# =============================================================================

if __name__ == "__main__":
    print("🔍 Starting data preparation with RELAXED filtering...")

    # Run the pipeline
    train, val, test = main_pipeline_with_relaxed_filtering()

    # Enhanced final verification
    print("\n🔍 ENHANCED FINAL VERIFICATION:")
    data_path = "/content/drive/MyDrive/financial_llm/data"

    # Multiple attempts to verify files
    for attempt in range(3):
        print(f"\n🔍 Verification attempt {attempt + 1}:")
        try:
            if os.path.exists(data_path):
                files = os.listdir(data_path)
                print(f"📁 Files in data directory: {files}")

                if files:
                    for file in files:
                        file_path = os.path.join(data_path, file)
                        size = os.path.getsize(file_path)
                        print(f"   📄 {file}: {size} bytes")
                    break  # Success, exit loop
                else:
                    print("   ℹ️ Directory exists but appears empty")
            else:
                print("❌ Data directory not found!")

            # Wait before retry
            if attempt < 2:
                time.sleep(3)

        except Exception as e:
            print(f"❌ Error during verification: {e}")
            if attempt < 2:
                time.sleep(3)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.8 MB/s eta 0:00:00
🔍 Starting data preparation with RELAXED filtering...
🚀 ENHANCED DATA PREPARATION PIPELINE WITH RELAXED FILTERING
🔧 ResumeManager initialized with base_path: /content/drive/MyDrive/financial_llm
🔧 Setting up Google Drive...
Mounted at /content/drive
✅ Google Drive mounted successfully
✅ Created/verified: /content/drive/MyDrive/financial_llm/data
✅ Created/verified: /content/drive/MyDrive/financial_llm/models
✅ Created/verified: /content/drive/MyDrive/financial_llm/results
🔧 EnhancedFinancialDataCollector initialized
🔧 GoogleDriveManager initialized with base_path: /content/drive/MyDrive/financial_llm

📥 STEP 1: Loading datasets...
🔍 Looking for checkpoint: /content/drive/MyDrive/financial_llm/data/checkpoint.json
ℹ️ No checkpoint found, starting from beginning
📥 Loading multiple finance datasets...
🔄 Loading finance-exam-data from 1rsh/finance-exam-data-generated...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/687k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

   ✅ Loaded 500 examples
   Converting finance-exam-data to common format...
   ✅ Converted 500 examples from finance-exam-data
✅ finance-exam-data: 500 examples after conversion
🔄 Loading finance-alpaca from gbharti/finance-alpaca...


...

   ✅ Loaded 68912 examples
   Converting finance-alpaca to common format...
   ✅ Converted 68912 examples from finance-alpaca
✅ finance-alpaca: 68912 examples after conversion
📊 Total data collected: 69412 examples from 2 datasets
💾 Saving checkpoint for stage: datasets_loaded
✅ Checkpoint saved: /content/drive/MyDrive/financial_llm/data/checkpoint.json

🎯 STEP 2: Enhancing dataset quality with RELAXED criteria...
🔍 Looking for checkpoint: /content/drive/MyDrive/financial_llm/data/checkpoint.json
✅ Checkpoint loaded from stage: datasets_loaded
🎯 Enhancing dataset quality with RELAXED criteria...
📊 Starting with 69412 raw examples


Relaxed Filtering: 100%|██████████| 69412/69412 [00:00<00:00, 185214.66it/s]


📊 After relaxed filtering: 65776 examples


Quality Scoring: 100%|██████████| 65776/65776 [00:01<00:00, 55109.67it/s]


📊 Current average score: 2.02
🎯 Target average score: 3.0
✅ Quality threshold 2.5 gives avg 3.41 with 26034 examples
📊 Final enhanced data: 26034 examples
💾 Saving checkpoint for stage: quality_enhanced
✅ Checkpoint saved: /content/drive/MyDrive/financial_llm/data/checkpoint.json

📊 STEP 3: Applying contest-optimized split...
🔍 Looking for checkpoint: /content/drive/MyDrive/financial_llm/data/checkpoint.json
✅ Checkpoint loaded from stage: quality_enhanced
🎯 Applying contest-optimized split (85-10-5)...
📊 Contest-optimized split:
   Training: 22128 examples (85.0%)
   Validation: 2603 examples (10.0%)
   Test: 1303 examples (5.0%)
💾 Saving checkpoint for stage: split_completed
✅ Checkpoint saved: /content/drive/MyDrive/financial_llm/data/checkpoint.json

💾 STEP 4: Saving to Google Drive...
💾 Saving datasets to Google Drive...
📁 Target directory: /content/drive/MyDrive/financial_llm/data
📄 Saving files:
   Train: /content/drive/MyDrive/financial_llm/data/contest_optimized_train.json
   

Below code creates subsets

In [ ]:
# CREATE REASONABLY SIZED SUBSETS FOR SFT TRAINING

import json
import random
import os
from google.colab import drive

# =============================================================================
# CONFIGURATION - ADJUST THESE VALUES AS NEEDED
# =============================================================================

# Percentage of data to keep (adjust based on your needs)
TRAIN_SAMPLE_PERCENT = 20    # 20% of training data (~4,400 examples)
VAL_SAMPLE_PERCENT = 50      # 50% of validation data (~1,300 examples)
TEST_SAMPLE_PERCENT = 50     # 50% of test data (~650 examples)

# Or specify exact number of examples (alternative to percentages)
# USE_EXACT_COUNTS = True
TRAIN_EXACT_COUNT = 5000     # Exact number of training examples
VAL_EXACT_COUNT = 1000       # Exact number of validation examples
TEST_EXACT_COUNT = 500       # Exact number of test examples

# Seed for reproducibility
RANDOM_SEED = 42

# File paths
BASE_PATH = "/content/drive/MyDrive/financial_llm/data"
ORIGINAL_TRAIN_FILE = f"{BASE_PATH}/contest_optimized_train.json"
ORIGINAL_VAL_FILE = f"{BASE_PATH}/contest_optimized_validation.json"
ORIGINAL_TEST_FILE = f"{BASE_PATH}/contest_optimized_test.json"

# Output file names
SUBSET_TRAIN_FILE = f"{BASE_PATH}/sft_subset_train.json"
SUBSET_VAL_FILE = f"{BASE_PATH}/sft_subset_validation.json"
SUBSET_TEST_FILE = f"{BASE_PATH}/sft_subset_test.json"
SUBSET_METADATA_FILE = f"{BASE_PATH}/sft_subset_metadata.json"

# =============================================================================
# SUBSET CREATION FUNCTION
# =============================================================================

def create_reasonable_subsets():
    """Create smaller, more manageable subsets of the datasets"""

    print("🔧 Creating reasonable-sized subsets for SFT training...")

    # Mount Google Drive if not already mounted
    try:
        drive.mount('/content/drive', force_remount=False)
        print("✅ Google Drive mounted")
    except:
        print("ℹ️ Google Drive already mounted")

    # Load original datasets
    print("\n📥 Loading original datasets...")

    with open(ORIGINAL_TRAIN_FILE, 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    print(f"✅ Loaded training data: {len(train_data)} examples")

    with open(ORIGINAL_VAL_FILE, 'r', encoding='utf-8') as f:
        val_data = json.load(f)
    print(f"✅ Loaded validation data: {len(val_data)} examples")

    with open(ORIGINAL_TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)
    print(f"✅ Loaded test data: {len(test_data)} examples")

    # Set random seed for reproducibility
    random.seed(RANDOM_SEED)

    # Calculate subset sizes
    print("\n📊 Calculating subset sizes...")

    # Using exact counts (comment out if you want to use percentages)
    train_sample_size = min(TRAIN_EXACT_COUNT, len(train_data))
    val_sample_size = min(VAL_EXACT_COUNT, len(val_data))
    test_sample_size = min(TEST_EXACT_COUNT, len(test_data))

    """
    # Alternative: Using percentages (uncomment if you prefer percentages)
    train_sample_size = int(len(train_data) * TRAIN_SAMPLE_PERCENT / 100)
    val_sample_size = int(len(val_data) * VAL_SAMPLE_PERCENT / 100)
    test_sample_size = int(len(test_data) * TEST_SAMPLE_PERCENT / 100)
    """

    print(f"🎯 Target sizes:")
    print(f"   Training: {train_sample_size} examples ({train_sample_size/len(train_data)*100:.1f}% of original)")
    print(f"   Validation: {val_sample_size} examples ({val_sample_size/len(val_data)*100:.1f}% of original)")
    print(f"   Test: {test_sample_size} examples ({test_sample_size/len(test_data)*100:.1f}% of original)")

    # Create subsets
    print("\n🎲 Creating random subsets...")

    train_subset = random.sample(train_data, train_sample_size)
    val_subset = random.sample(val_data, val_sample_size)
    test_subset = random.sample(test_data, test_sample_size)

    print(f"✅ Created subsets:")
    print(f"   Training: {len(train_subset)} examples")
    print(f"   Validation: {len(val_subset)} examples")
    print(f"   Test: {len(test_subset)} examples")

    # Save subsets
    print("\n💾 Saving subsets to Google Drive...")

    with open(SUBSET_TRAIN_FILE, 'w', encoding='utf-8') as f:
        json.dump(train_subset, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved training subset: {SUBSET_TRAIN_FILE}")

    with open(SUBSET_VAL_FILE, 'w', encoding='utf-8') as f:
        json.dump(val_subset, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved validation subset: {SUBSET_VAL_FILE}")

    with open(SUBSET_TEST_FILE, 'w', encoding='utf-8') as f:
        json.dump(test_subset, f, indent=2, ensure_ascii=False)
    print(f"✅ Saved test subset: {SUBSET_TEST_FILE}")

    # Create and save metadata
    metadata = {
        'subset_creation_timestamp': pd.Timestamp.now().isoformat(),
        'random_seed': RANDOM_SEED,
        'original_sizes': {
            'train': len(train_data),
            'validation': len(val_data),
            'test': len(test_data)
        },
        'subset_sizes': {
            'train': len(train_subset),
            'validation': len(val_subset),
            'test': len(test_subset)
        },
        'sampling_percentages': {
            'train': len(train_subset)/len(train_data)*100,
            'validation': len(val_subset)/len(val_data)*100,
            'test': len(test_subset)/len(test_data)*100
        },
        'sampling_method': 'random',
        'purpose': 'SFT_training_reasonable_size'
    }

    with open(SUBSET_METADATA_FILE, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Saved subset metadata: {SUBSET_METADATA_FILE}")

    # Verify file sizes
    print("\n🔍 Verifying saved files:")
    for file_path in [SUBSET_TRAIN_FILE, SUBSET_VAL_FILE, SUBSET_TEST_FILE]:
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path)
            file_size_mb = file_size / (1024 * 1024)
            print(f"   📄 {os.path.basename(file_path)}: {file_size:,} bytes ({file_size_mb:.1f} MB)")
        else:
            print(f"   ❌ {os.path.basename(file_path)}: NOT FOUND")

    print("\n✅ SUBSET CREATION COMPLETED SUCCESSFULLY!")
    print(f"🎯 Final subset sizes perfect for SFT:")
    print(f"   Training: {len(train_subset):,} examples")
    print(f"   Validation: {len(val_subset):,} examples")
    print(f"   Test: {len(test_subset):,} examples")
    print(f"   Total: {len(train_subset) + len(val_subset) + len(test_subset):,} examples")

    return train_subset, val_subset, test_subset

# =============================================================================
# QUALITY-BASED SUBSET (ALTERNATIVE - KEEPS HIGHEST QUALITY EXAMPLES)
# =============================================================================

def create_quality_based_subsets(top_percentage=30):
    """Create subsets by keeping only the highest quality examples"""

    print(f"🎯 Creating quality-based subsets (top {top_percentage}%)...")

    # Mount Google Drive
    try:
        drive.mount('/content/drive', force_remount=False)
    except:
        pass

    # Load original datasets
    with open(ORIGINAL_TRAIN_FILE, 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    with open(ORIGINAL_VAL_FILE, 'r', encoding='utf-8') as f:
        val_data = json.load(f)
    with open(ORIGINAL_TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

    print(f"📊 Original dataset sizes:")
    print(f"   Training: {len(train_data)} examples")
    print(f"   Validation: {len(val_data)} examples")
    print(f"   Test: {len(test_data)} examples")

    # Sort by quality score and take top percentage
    def get_top_examples(data, percentage):
        sorted_data = sorted(data, key=lambda x: x.get('conceptual_score', 0), reverse=True)
        keep_count = int(len(sorted_data) * percentage / 100)
        return sorted_data[:keep_count]

    train_quality = get_top_examples(train_data, top_percentage)
    val_quality = get_top_examples(val_data, top_percentage)
    test_quality = get_top_examples(test_data, top_percentage)

    print(f"📊 Quality-based subset sizes (top {top_percentage}%):")
    print(f"   Training: {len(train_quality)} examples")
    print(f"   Validation: {len(val_quality)} examples")
    print(f"   Test: {len(test_quality)} examples")

    # Calculate average quality scores
    def avg_quality(data):
        scores = [item.get('conceptual_score', 0) for item in data]
        return sum(scores) / len(scores) if scores else 0

    print(f"📈 Average quality scores:")
    print(f"   Training: {avg_quality(train_quality):.2f}")
    print(f"   Validation: {avg_quality(val_quality):.2f}")
    print(f"   Test: {avg_quality(test_quality):.2f}")

    # Save quality-based subsets
    quality_train_file = f"{BASE_PATH}/sft_quality_train.json"
    quality_val_file = f"{BASE_PATH}/sft_quality_validation.json"
    quality_test_file = f"{BASE_PATH}/sft_quality_test.json"

    with open(quality_train_file, 'w', encoding='utf-8') as f:
        json.dump(train_quality, f, indent=2, ensure_ascii=False)
    with open(quality_val_file, 'w', encoding='utf-8') as f:
        json.dump(val_quality, f, indent=2, ensure_ascii=False)
    with open(quality_test_file, 'w', encoding='utf-8') as f:
        json.dump(test_quality, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved quality-based subsets:")
    print(f"   Training: {quality_train_file}")
    print(f"   Validation: {quality_val_file}")
    print(f"   Test: {quality_test_file}")

    return train_quality, val_quality, test_quality

# =============================================================================
# DATASET ANALYZER (HELPER FUNCTION)
# =============================================================================

def analyze_datasets():
    """Analyze the datasets to help decide on reasonable sizes"""

    print("📊 DATASET ANALYSIS REPORT")
    print("=" * 50)

    # Load datasets
    with open(ORIGINAL_TRAIN_FILE, 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    with open(ORIGINAL_VAL_FILE, 'r', encoding='utf-8') as f:
        val_data = json.load(f)
    with open(ORIGINAL_TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)

    # Basic stats
    print(f"Dataset sizes:")
    print(f"  Training: {len(train_data):,} examples")
    print(f"  Validation: {len(val_data):,} examples")
    print(f"  Test: {len(test_data):,} examples")
    print(f"  Total: {len(train_data) + len(val_data) + len(test_data):,} examples")

    # Quality score distribution
    def get_quality_stats(data, name):
        scores = [item.get('conceptual_score', 0) for item in data]
        if scores:
            return {
                'name': name,
                'avg_score': sum(scores) / len(scores),
                'min_score': min(scores),
                'max_score': max(scores),
                'count': len(scores)
            }
        return None

    train_stats = get_quality_stats(train_data, 'Training')
    val_stats = get_quality_stats(val_data, 'Validation')
    test_stats = get_quality_stats(test_data, 'Test')

    print(f"\nQuality Score Analysis:")
    for stats in [train_stats, val_stats, test_stats]:
        if stats:
            print(f"  {stats['name']}: avg={stats['avg_score']:.2f}, min={stats['min_score']:.1f}, max={stats['max_score']:.1f}")

    # Sample sizes for different scenarios
    print(f"\n💡 Recommended subset sizes for SFT:")
    print(f"  Small experiment: 1K train, 200 val, 100 test")
    print(f"  Medium project: 5K train, 1K val, 500 test")
    print(f"  Large project: 10K train, 2K val, 1K test")
    print(f"  Full scale: 20K+ train, 2K+ val, 1K+ test")

    # File size estimates
    def estimate_file_size(data):
        # Rough estimate: ~1KB per example on average
        return len(data) * 1024

    print(f"\n📁 File size estimates:")
    print(f"  Current training: {estimate_file_size(train_data)/ (1024*1024):.1f} MB")
    print(f"  5K training subset: ~{5 * 1024 / 1024:.1f} MB")
    print(f"  10K training subset: ~{10 * 1024 / 1024:.1f} MB")

# =============================================================================
# EXECUTION
# =============================================================================

if __name__ == "__main__":
    import pandas as pd

    print("🎯 CREATING REASONABLE-SIZED SUBSETS FOR SFT")
    print("=" * 60)

    # First, analyze the datasets
    analyze_datasets()

    print("\n" + "=" * 60)
    print("🚀 Creating random subsets with exact counts...")

    # Create random subsets (recommended for most use cases)
    train_subset, val_subset, test_subset = create_reasonable_subsets()

    print("\n" + "=" * 60)
    print("🌟 OPTIONAL: Creating quality-based subsets...")

    # Uncomment if you want quality-based subsets instead
    # train_quality, val_quality, test_quality = create_quality_based_subsets(top_percentage=30)

    print("\n✅ ALL SUBSETS CREATED SUCCESSFULLY!")
    print("\n📁 Your new subset files:")
    print(f"   {SUBSET_TRAIN_FILE}")
    print(f"   {SUBSET_VAL_FILE}")
    print(f"   {SUBSET_TEST_FILE}")
    print(f"   {SUBSET_METADATA_FILE}")

    print("\n🎯 These subsets are perfect for SFT training!")

🎯 CREATING REASONABLE-SIZED SUBSETS FOR SFT
📊 DATASET ANALYSIS REPORT
Dataset sizes:
  Training: 22,128 examples
  Validation: 2,603 examples
  Test: 1,303 examples
  Total: 26,034 examples

Quality Score Analysis:
  Training: avg=3.41, min=2.5, max=7.0
  Validation: avg=3.42, min=2.5, max=6.0
  Test: avg=3.41, min=2.5, max=6.0

💡 Recommended subset sizes for SFT:
  Small experiment: 1K train, 200 val, 100 test
  Medium project: 5K train, 1K val, 500 test
  Large project: 10K train, 2K val, 1K test
  Full scale: 20K+ train, 2K+ val, 1K+ test

📁 File size estimates:
  Current training: 21.6 MB
  5K training subset: ~5.0 MB
  10K training subset: ~10.0 MB

🚀 Creating random subsets with exact counts...
🔧 Creating reasonable-sized subsets for SFT training...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted

📥 Loading original datasets...
✅ Loaded training data: 22128 examples
✅ Loaded va